In [9]:
#pastas colocadas e que funcionam:
#seguenotasfiscais2quinzenadedezembropnz funcionando<<
#n_f52transf_jpajabjpa (deve funcionar mas só 1 file)
#n_fs1718jaymejosedelimajunio (deve funcionar mas só 2 file)
#nfs_henrique_roberval_josrogrio funcionando<<
#nfs_jarbiana não_funcionando <<<<<<<<MAS TA LENDO>>>>>>>>
#nfsref2quinzenadezembro2023 tem que ser feito com o leitor de imagem
#ren_fs535455e56edvaldodesouzasilvajunior funcionando <<<

In [8]:
def leitor(dir):
    import os
    import csv  
    import pdfplumber
    import pandas as pd
    
    csv_file_path = dir + '/pdf_analise.csv'
    colunas = ('chave','cnpj_prestador','cnpj_tomador','valor_servico','numero_nota','data','email_tomador','email_prestador','empresa_tomador','empresa_prestador','codigo_tributacao')            
    with open(csv_file_path,'w',newline='') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(colunas)
        
    fileExt = r".pdf"
    files = os.listdir(dir)
    pdf_files = [arq for arq in files if arq.lower().endswith('.pdf')]
    for pdf_file in pdf_files:
        with pdfplumber.open(f"{dir}/{pdf_file}") as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
                a = text
                lines = a.split('\n')
                cda=False
                tds=False
                vds=False
                nfs=False
                ne=False
                em=0
                cdt=False
                counter=0
                for line in lines:
                    if cdt==True:
                        counter=counter+1
                        if counter==1:
                            codigo_tributacao=line.split(' ')[0].split('-')[0]
                            desc1=line.split(' ')[1]
                            
                        if counter==2:
                            desc2=line.split(' ')[0]
                            desc=desc1+desc2
                            cdt=False
                        cdt=cdt+1
                        
                    if ne==True:
                        if em == 0:
                            email_prestador=line.split(' ')[1]
                            empresa_tomador = line.split('-')[0]
                            em =1
                        email_tomador=line.split(' ')[1]
                        empresa_prestador = line.split(' ')[0]
                        ne=False
                    if nfs == True:
                        numero_nota= line.split(' ')[0]
                        data = line.split(' ')[1]
                        nfs=False
                    if vds == True:
                        valor_servico=line.split(' ')[0]
                        vds=False
                    if tds== True:
                        cnpj_tomador = line.split(' ')[0]
                        tds = False
                    if cda == True:
                        chave = line.replace(' ','')[0:44]
                        cda=False
                    if 'ChavedeAcessodaNFS' in line:
                        cda=True
                    
                    if 'PrestadordoServiço' in line:
                        cnpj_prestador = line.split(' ')[1]
                    if 'TOMADORDOSERVIÇO' in line:
                        tds=True
                        
                    if 'ValordoServiço' in line:
                        vds=True
                        
                    if 'NúmerodaNFS' in line:
                        nfs=True
                        
                    if 'Nome/NomeEmpresarial' in line:
                        ne=True
                        
                    if 'CódigodeTributaçãoNacional' in line:
                        cdt=True
                with open(csv_file_path, mode='a', encoding= 'utf8',newline='') as resultado_pesquisa:
                    writer = csv.writer(resultado_pesquisa)
                    linhas = (chave,cnpj_prestador,cnpj_tomador,valor_servico,numero_nota,data,email_tomador,email_prestador,empresa_tomador,empresa_prestador,codigo_tributacao)
                    writer.writerow(linhas)
                
                #(r"C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz/LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf")
            


In [17]:
import pdfplumber
cont = 0
with pdfplumber.open(r"C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz/LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf") as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
                a = text
                lines = a.split('\n')
                cda=False
                tds=False
                vds=False
                nfs=False
                ne=False
                em=0
                cdt=False
                counter=0
                for line in lines:
                    if cdt==True:
                        counter=counter+1
                        if counter==1:
                            codigo_tributacao=line.split(' ')[0].split('-')[0]
                            desc1=line.split(' ')[1]
                            
                        if counter==2:
                            desc2=line.split(' ')[0]
                            desc=desc1+desc2
                            cdt=False
                        cdt=cdt+1
                        
                    if ne==True:
                        if em == 0:
                            email_prestador=line.split(' ')[1]
                            empresa_tomador = line.split('-')[0]
                            em =1
                        email_tomador=line.split(' ')[1]
                        empresa_prestador = line.split(' ')[0]
                        ne=False
                    if nfs == True:
                        numero_nota= line.split(' ')[0]
                        data = line.split(' ')[1]
                        nfs=False
                    if vds == True:
                        valor_servico=line.split(' ')[0]
                        vds=False
                    if tds== True:
                        cnpj_tomador = line.split(' ')[0]
                        tds = False
                    if cda == True:
                        chave = line.replace(' ','')[0:44]
                        cda=False
                    if 'ChavedeAcessodaNFS' in line:
                        cda=True
                    
                    if 'PrestadordoServiço' in line:
                        cnpj_prestador = line.split(' ')[1]
                    if 'TOMADORDOSERVIÇO' in line:
                        tds=True
                        
                    if 'ValordoServiço' in line:
                        vds=True
                        
                    if 'NúmerodaNFS' in line:
                        nfs=True
                        
                    if 'Nome/NomeEmpresarial' in line:
                        ne=True
                        
                    if 'CódigodeTributaçãoNacional' in line:
                        cdt=True
                    print(line,cont)
            print(chave,cnpj_prestador,cnpj_tomador,valor_servico,numero_nota,data,email_tomador,email_prestador,empresa_tomador,empresa_prestador,codigo_tributacao)
            

NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFSe 0
MUNICIPIO DE PETROLINA 0
Codigo de Verificação para Autenticação: 22bd2510d 0
Endereço: Petrolina, Pernambuco, PE, 56302-000 0
CNPJ: 10.358.190/0001-77, E-mail: GABINETEPREFEITURAPETROLINA@HOTMAIL.COM 0
Emitido em 02/01/2024 08:38:20 0
Data Fato Gerador Exigibilidade de ISS Regime Tributário Número RPS Nº da Nota Fiscal 0
02/01/2024 Exigível Tributacao Normal 0
119 0
Tipo de Recolhimento Simples Local de Prestação Local de Recolhimento 0
Não Retido Optante 2611101 - Petrolina - PE 2611101 - Petrolina - PE 0
PRESTADOR 0
Razão Social: LAZARO JOEDSON MOURA SILVA 0
Nome Fantasia: MILHAO DO VALE 0
Endereço: Rua NUCLEO RURAL PSNC - N7, 115, LETRA D - ZONA RURAL 0
Petrolina - PE - CEP: 56302-970 0
E-mail: contabilidadedovale@hotmail.com - Fone: - Site: ........ 0
Inscrição Estadual: ........ - Inscrição Municipal: 9632190 - CPF/CNPJ: 24.254.072/0001-23 0
TOMADOR 0
Razão Social: PROGRESSO LOGISTICA E TRANSPORTE LTDA 0
Endereço: Rua SETE DE SETEMBRO, 

In [25]:
leitor('C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz')

26111012236240873000119000000000000924018790 36.240.873/0001-19 10.580.938/0005-05 R$900,00 9 02/01/2024 - NENEBENE1@HOTMAIL.COM BERNEVALDORIBEIROGOMES74792350468 NENEBENE1@HOTMAIL.COM PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
26111012252685890000116000000000000924016396 52.685.890/0001-16 10.580.938/0005-05 R$540,00 9 02/01/2024 - DANIELPNZVIDROS@GMAIL.COM 52.685.890DANIELRODRIGUESSILVABEZERRA DANIELPNZVIDROS@GMAIL.COM PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
26111012252685890000116000000000000824013952 52.685.890/0001-16 10.580.938/0005-05 R$2.700,00 8 02/01/2024 - DANIELPNZVIDROS@GMAIL.COM 52.685.890DANIELRODRIGUESSILVABEZERRA DANIELPNZVIDROS@GMAIL.COM PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
26111012252685890000116000000000000824013952 52.685.890/0001-16 10.580.938/0005-05 R$2.700,00 8 02/01/2024 - DANIELPNZVIDROS@GMAIL.COM 52.685.890DANIELRODRIGUESSILVABEZERRA DANIELPNZVIDROS@GMAIL.COM PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
26111012252685890000116000000000000824013952 52.

In [26]:
leitor(r'C:\Users\jvict\Documents\seven\seven\n_f52transf_jpajabjpa')

25075072223159548000184000000000005224014953 23.159.548/0001-84 10.580.938/0010-72 R$2.808,00 52 02/01/2024 EDUARDO@HECONTABIL.COM.BR - EDVALDODESOUZASILVAJUNIOR07562256470  PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01


In [9]:

leitor('C:/Users/jvict/Documents/seven/seven/nfs_henrique_roberval_josrogrio')


In [5]:
dir = r"C:\Users\jvict\Documents\seven\seven\nfs_jarbiana\654.pdf"
with pdfplumber.open(dir)as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
                a = text
                lines = a.split('\n')
                for line in lines:
                    print(line)

PREFEITURA DE CARUARU Número da
SECRETARIA DE NEGOCIOS DA FAZENDA NFS-e
NOTA FISCAL ELETRÔNICA DE SERVIÇO - NFS-e 654
Código de Verificação
Data e Hora da Emissão 02/01/2024 13:56:10 Competência 2/1/2024 1XGSXWOYN
No. da NFS-e substituída
Número do RPS Local da Prestação CARUARU - PE
Dados do Prestador de Serviços
Razão Social/Nome JARBIANA CRISTINA DE ASSIS B SILVA
Nome Fantasia
CNPJ/CPF 33.037.636/0001-67 Inscrição Municipal 90609971 Município CARUARU - PE
Endereço e Cep Rua Independência ,8 - Kennedy CEP: 55036-180
Complemento: Telefone: 81995550812 e-mail: multicontape@gmail.com
Dados do Tomador de Serviços
Razão Social/Nome PROGRESSO LOGISTICA E TRANSPORTE LTDA
CNPJ/CPF 10.580.938/0001-81 Inscrição Municipal Município JABOATAO DOS GUARARAPES - PE
Endereço e CEP RUA OITENTA ,100 - CURADO CEP: 54270-165
Complemento: QD - B; LT- 01; A - 08; Telefone: (81)2121-9000 e-mail: SECRETARIA@ADTSA.COM.BR
Discriminação dos Serviços
Referente a 1 viagem Jab x Mcz x Jab, 1 viagem Jab x Nat x Jab

In [29]:
leitor(r"C:\Users\jvict\Documents\seven\seven\ren_fs535455e56edvaldodesouzasilvajunior")

25075072223159548000184000000000005324017802 23.159.548/0001-84 10.580.938/0010-72 R$1.530,00 53 02/01/2024 EDUARDO@HECONTABIL.COM.BR - EDVALDODESOUZASILVAJUNIOR07562256470  PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
25075072223159548000184000000000005424012535 23.159.548/0001-84 10.580.938/0010-72 R$100,00 54 02/01/2024 EDUARDO@HECONTABIL.COM.BR - EDVALDODESOUZASILVAJUNIOR07562256470  PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
25075072223159548000184000000000005524010476 23.159.548/0001-84 10.580.938/0010-72 R$2.970,00 55 02/01/2024 EDUARDO@HECONTABIL.COM.BR - EDVALDODESOUZASILVAJUNIOR07562256470  PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
25075072223159548000184000000000005624011233 23.159.548/0001-84 10.580.938/0010-72 R$330,00 56 02/01/2024 EDUARDO@HECONTABIL.COM.BR - EDVALDODESOUZASILVAJUNIOR07562256470  PROGRESSOLOGISTICAETRANSPORTELTDA 16.02.01
